In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install --upgrade scikit-learn
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('bmh')
import seaborn as sns
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split,StratifiedKFold,GridSearchCV
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor,AdaBoostRegressor,BaggingRegressor,ExtraTreesRegressor,VotingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.ensemble import VotingRegressor
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.metrics import mean_squared_log_error
%matplotlib inline

from sklearn.preprocessing import MaxAbsScaler
from sklearn.inspection import permutation_importance

from xgboost import XGBRegressor

from scipy.stats import skew

from scipy import stats

In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
df=pd.read_csv('/kaggle/input/bike-sharing-demand/train.csv',parse_dates=['datetime'])
x_test=pd.read_csv('/kaggle/input/bike-sharing-demand/test.csv',parse_dates=['datetime'])
display(df.head())

In [ ]:
print(df.shape)
print(x_test.shape)

In [ ]:
display(df.describe().T)
display(df.info())

In [ ]:
plt.figure(figsize=(25, 25))
for i, col in enumerate(list(df.columns)):
    plt.subplot(7, 4, i+1)
    sns.histplot(df[col], kde=True)

In [ ]:
def insights(df):
    for col in df.columns:
        if col=='count':
            continue
        else:
            display(df[[col, 'count']].groupby([col], as_index=False).mean().sort_values(by='count', ascending=False).T)

In [ ]:
insights(df)

In [ ]:
df.nunique()

In [ ]:
def visualize_counts(df):
    low_unique_values_cols=['season','holiday','workingday']
    for col in low_unique_values_cols:
        sns.catplot(x=col, kind="count", palette="ch:.25", data=df)
        plt.title("Plotting "+ str(col))

In [ ]:
visualize_counts(df)

In [ ]:
df['Year'] = df['datetime'].dt.year
df['Month'] = df['datetime'].dt.month
df['Day'] = df['datetime'].dt.day
df['hour'] = df['datetime'].dt.hour
df['weekday'] = df['datetime'].dt.dayofweek
df['dayofyear']=df['datetime'].dt.dayofyear

x_test['Year'] = x_test['datetime'].dt.year
x_test['Month'] = x_test['datetime'].dt.month
x_test['Day'] = x_test['datetime'].dt.day
x_test['hour'] = x_test['datetime'].dt.hour
x_test['weekday'] = x_test['datetime'].dt.dayofweek
x_test['dayofyear']=x_test['datetime'].dt.dayofyear


In [ ]:
sns.barplot(x='hour',y='count',data=df)
plt.show()

In [ ]:
df['rush Hours'] = df['hour'].isin([8,16,17,18,19])
x_test['rush Hours'] = x_test['hour'].isin([8,16,17,18,19])

In [ ]:
fig, ax = plt.subplots(figsize=(14,8))
cor_mat= df[:].corr()
mask = np.array(cor_mat)
mask[np.tril_indices_from(mask)] = False
fig=plt.gcf()
fig.set_size_inches(15,10)
sns.heatmap(data=cor_mat,mask=mask,square=True,annot=True,cbar=True)
plt.title("Columns Correlations");

In [ ]:
sns.catplot(x="temp",y="count",data=df,kind='bar',height=6,aspect=3)

In [ ]:
new_df=df.copy()
new_df['temp_bin']=np.floor(new_df['temp'])//5
new_df['temp_bin'].unique()
sns.catplot(x="temp_bin",y="count",data=new_df,kind='bar',height=5,aspect=1.5)

In [ ]:
df['temp_bin']=np.floor(df['temp'])//5
x_test['temp_bin']=np.floor(x_test['temp'])//5

In [ ]:
df['count']=np.log1p(df['count'])

In [ ]:
Y=df['count']
df.drop(columns=['datetime','count','casual','registered'],inplace = True)
x_test_index=x_test['datetime']
x_test.drop(columns=['datetime'],inplace = True)

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(df,Y, train_size=0.8, test_size=0.2)

In [ ]:
estimators=[('et',ExtraTreesRegressor()),('hgr', HistGradientBoostingRegressor())]
models=[RandomForestRegressor(),AdaBoostRegressor(),BaggingRegressor(),LinearRegression(),DecisionTreeRegressor(),ExtraTreesRegressor(), HistGradientBoostingRegressor(),VotingRegressor(estimators=estimators)]
model_names=['RandomForestRegressor','AdaBoostRegressor','BaggingRegressor','LinearRegression','DecisionTreeRegressor','ExtraTreesRegressor',' HistGradientBoostingRegressor','VotingRegressor']
rmsle=[]
d={}
for model in range (len(models)):
    clf=models[model]
    clf.fit(X_train,y_train)
    print("model_name : ",model_names[model])
    print(clf.get_params())
    test_pred=clf.predict(X_valid)
    #print(test_pred)
    rmsle.append(np.sqrt(mean_squared_log_error(abs(test_pred),(y_valid))))
    
d={'Modelling Algo':model_names,'RMSLE':rmsle}   
d

In [ ]:
rmsle_frame=pd.DataFrame(d)
rmsle_frame

In [ ]:
sns.catplot(y='Modelling Algo',x='RMSLE',data=rmsle_frame,kind='bar',height=5,aspect=2)


In [ ]:
def rmsle(y_pred,y_true):
    y_pred = np.expm1(y_pred)
    y_true = np.expm1(y_true)
    log1=np.log(y_pred + 1)
    log2=np.log(y_true + 1)
    se = (log1 - log2) ** 2 
    mse=np.mean(se)
    return np.sqrt(mse)

from sklearn.metrics import make_scorer
myScorer = make_scorer(rmsle, greater_is_better=False)

In [ ]:
et = HistGradientBoostingRegressor()

param = {'max_iter' : [365], 
         'max_depth':[15,16,17],

         'warm_start':[True],
         #min_samples_split':[2],
         #min_samples_leaf':[1],
        }
gridSearch_ExtraTrees = GridSearchCV(et,param,scoring=myScorer,cv=5,verbose=3)
gridSearch_ExtraTrees.fit(X_train, y_train)

best_ExtraTrees = gridSearch_ExtraTrees.best_estimator_
bestExtraTrees_testScore=best_ExtraTrees.score(X_train, y_train)

In [ ]:
gridSearch_ExtraTrees.best_params_

In [ ]:
pred=best_ExtraTrees.predict(X_valid)
print(rmsle(pred,y_valid))

In [ ]:
model = HistGradientBoostingRegressor(max_depth=16,max_iter=265,warm_start=True)
model.fit(df,Y)

In [ ]:
# features=df.columns
# importances = model.feature_importances_
# indices = np.argsort(importances)
# plt.figure(figsize=(10,15))
# plt.title('Feature Importances')
# plt.barh(range(len(indices)), importances[indices], color='b', align='center')
# plt.yticks(range(len(indices)), [features[i] for i in indices])
# plt.xlabel('Relative Importance')
# plt.show()

In [ ]:
r = permutation_importance(best_ExtraTrees, X_valid, y_valid,
                            n_repeats=30)

for i in r.importances_mean.argsort()[::-1]:

    print(f"{X_train.columns[i]} "
           f"{r.importances_mean[i]:.3f} "
           f" +/- {r.importances_std[i]:.3f}")

In [ ]:
plt.figure(figsize=(10,7))
plt.barh(X_train.columns, r.importances_mean)

In [ ]:
x_test.head()

In [ ]:
X_train.head()

In [ ]:
pred=np.round(np.expm1(best_ExtraTrees.predict(x_test))).astype(int)
predictions = pd.DataFrame({'datetime':x_test_index,
                       'count': pred})

In [ ]:
predictions.to_csv('submission.csv', index=False)
print("Your submission was successfully saved!")